In [1]:
import metapy
import math
idx = metapy.index.make_inverted_index('config.toml')


In [2]:
# Examine number of documents
print idx.num_docs()
# Number of unique terms in the dataset
print idx.unique_terms()
# The average document length
print idx.avg_doc_length()
# The total number of terms
print idx.total_corpus_terms()

1
1179
5824.0
5824


In [5]:
# Build the query object and initialize a ranker
query = metapy.index.Document()
ranker = metapy.index.JelinekMercer(.0)
# To do an IR evaluation, we need to use the queries file and relevance judgements.
ev = metapy.index.IREval('config.toml')
# We will loop over the queries file and add each result to the IREval object ev.
num_results = 10
with open('cranfield-queries.txt') as query_file:
    for query_num, line in enumerate(query_file):
        query.content(line.strip())
        results = ranker.score(idx, query, num_results)                            
        avg_p = ev.avg_p(results, query_num, num_results)
        #print("Query {} average precision: {}".format(query_num + 1, avg_p))
ev.map()

0.0414943982531284

In [67]:
class PL2Ranker(metapy.index.RankingFunction):                                            
    """ Create a new ranking function in Python that can be used in MeTA. """                                                                          
    def __init__(self, some_param=1.0):                                             
        self.param = some_param
        # You *must* call the base class constructor here!
        super(PL2Ranker, self).__init__()                                        

    def score_one(self, sd):
        """ You need to override this function to return a score for a single term. For fields available in the score_data sd object, @see https://meta-toolkit.org/doxygen/structmeta_1_1index_1_1score__data.html """
        tfn = sd.doc_term_count + math.log(1 + self.param * (sd.avg_dl / sd.doc_size))
        lam = 1.0 * sd.num_docs / sd.corpus_term_count
        
        if tfn <= 0 or lam < 1:
            return 0
        
        term1 = tfn * math.log(tfn * lam, 2.0)
        term2 = math.log(math.e, 2.0 ) * (1.0 / lam - tfn)
        term3 = 0.5 * math.log(2.0 * math.pi * tfn, 2.0)
        n = term1 + term2 + term3
        return sd.query_term_weight * n / (tfn + 1.0)

In [79]:
# Build the query object and initialize a ranker
query = metapy.index.Document()
ranker = PL2Ranker(.0)
# To do an IR evaluation, we need to use the queries file and relevance judgements.
ev = metapy.index.IREval('config.toml')
# We will loop over the queries file and add each result to the IREval object ev.
num_results = 10
with open('cranfield-queries.txt') as query_file:
    for query_num, line in enumerate(query_file):
        query.content(line.strip())
        results = ranker.score(idx, query, num_results)                            
        avg_p = ev.avg_p(results, query_num, num_results)
        #print("Query {} average precision: {}".format(query_num + 1, avg_p))
ev.map()

0.03436672545561434

## 